In [1]:
from image_prep import get_img_df, train_val_test_split
from image_gen import get_gen_from_df

In [2]:
fc_path = "data/Food_Classification/"

# get df
df_fc = get_img_df(fc_path)

num_classes = len(df_fc['label'].unique())
num_classes

20

In [3]:
# split dfs
train, validate, test = train_val_test_split(df_fc, test_size=0.2, val_size=0.2)

In [4]:
train_gen = get_gen_from_df(train)
validate_gen = get_gen_from_df(validate, train=False)

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 classes.


c:\Users\Jon\miniconda3\envs\Tasty_AI\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


The following model is probably not the final model we want to use but just needed something to test and end to end flow.  

This particular architecture was borrowed from the Jupyter notebook found: https://www.kaggle.com/code/varsha300/transferlearning


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Constants
IMG_SIZE = (224, 224)  # VGG16 default image size

# model

base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
# Freeze base model layers and unfreeze the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Model architecture
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Train model
history = model.fit(train_gen, validation_data=validate_gen, epochs=1, steps_per_epoch=train_gen.num_batches)


c:\Users\Jon\Documents\UMSI\Courses\dev\Capstone\indian-food\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\Jon\Documents\UMSI\Courses\dev\Capstone\indian-food\.venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 360s 3s/step - accuracy: 0.0682 - loss: 3.4872 - val_accuracy: 0.3012 - val_loss: 2.5296


In [7]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validate_gen)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

40/40 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.3020 - loss: 2.5210
Validation accuracy: 30.12%
